In [ ]:
import scanpy as sc
import scvelo as scv
from glob import glob
import pandas as pd
import numpy as np
import seaborn as sns
import anndata
import scipy
import re
import os
import matplotlib
import math
import random
import itertools
import gseapy as gp
import sklearn
from icecream import ic
from statannot import add_stat_annotation
from matplotlib import pyplot as plt
from matplotlib import rcParams
from matplotlib.legend import Legend
import matplotlib.gridspec as gridspec

import generalfunctions as gf
import populationfunctions as pf
import airrfunctions as airr
import dgexfunctions as dgexfunc

%matplotlib inline
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 200)
pd.options.display.max_seq_items = 2000

sc.set_figure_params(scanpy=True, dpi=300, dpi_save=300, frameon=True, vector_friendly=True, fontsize=12, 
                         color_map='Dark2', format='pdf', transparent=True, ipython_format='png2x')

rcParams.update({'font.size': 8})
rcParams.update({'font.family': 'Helvetica'})
rcParams['pdf.fonttype'] = 42
rcParams['ps.fonttype'] = 42
rcParams['svg.fonttype'] = 'none'
rcParams['figure.facecolor'] = (1,1,1,1)

import warnings
warnings.filterwarnings("ignore")

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.core.display import display, HTML
display(HTML("""
<style>
#notebook-container {
    width: 100%
}
 
.code_cell {
   flex-direction: row !important;
}
 
.code_cell .input {
    width: 50%
}
 
.code_cell .output_wrapper {
    width: 50%
}
</style>
"""))

In [ ]:
## load celltype data

embFiles = {'tcell_filtered':'umap_n-0055_md-0.80_s-2.28.npy',
            'bcell_filtered_2':'umap_n-0037_md-0.40_s-0.50.npy',
            'myeloid_filtered':'umap_n-0064_md-0.10_s-1.61.npy'}

clustFiles = {'tcell_filtered':'scvi_cugraph_leiden_nbr100_res0.6.npy',
           'bcell_filtered_2':'scvi_cugraph_leiden_nbr100_res0.6.npy',
           'myeloid_filtered':'scvi_cugraph_leiden_nbr100_res1.0.npy'}


celltype = 'bcell_filtered_2'
path = '/Users/gouink/Documents/RTPD1Manuscript/Human/manuscript_review_analysis/scvi_outputs/'

adata = sc.read_h5ad('/Users/gouink/Documents/RTPD1Manuscript/Human/manuscript_review_analysis/h5adfiles/PembroRT_immune_R100.h5ad')

BC = np.load(path+'/'+celltype+'/barcodes.npy',allow_pickle=True)
adata = adata[list(BC)].copy()
adata

emb = np.load(path+'/'+celltype+'/'+embFiles[celltype])
clust = np.load(path+'/'+celltype+'/'+clustFiles[celltype])

adata.obsm['X_umap'] = emb
adata.obs['leiden'] = [str(x) for x in clust]
adata.obs.leiden = adata.obs.leiden.astype('category')

metadata = pd.read_csv('/Users/gouink/Documents/RTPD1Manuscript/Human/PEMBRORT_CLINICAL_METADATA_FORSCSEQ_KHG20201119.csv',index_col=None,header=0)
cols = [x for x in metadata.columns if x not in adata.obs.columns]
metadata = metadata[cols]
adata.obs = adata.obs.reset_index().merge(metadata,left_on='cohort',right_on='Patient_Number',how='left').set_index('index')
# adata.obs.to_csv('/Users/gouink/Documents/RTPD1Manuscript/Human/manuscript_review_analysis/metadata_collection/'+celltype+'_obs.csv')

dotsize = (120000/len(adata))*2

sc.pl.umap(adata,color='leiden',size=dotsize,show=False)
# plt.savefig('/Users/gouink/Documents/RTPD1Manuscript/Human/manuscript_review_analysis/analysis/'+celltype+'/umap_by_leiden.png',dpi=600,bbox_inches='tight')

adata.raw = adata
sc.pp.normalize_per_cell(adata, counts_per_cell_after=10000)
sc.pp.log1p(adata)

# dgex = dgexfunc.leiden_dgex(adata)
# dgex.to_csv('/Users/gouink/Documents/RTPD1Manuscript/Human/manuscript_review_analysis/analysis/'+celltype+'/leiden_dgex.csv')

In [ ]:
## some key markers

GOI = ['CD79A','CD79B','CD19','MS4A1','CD40','CXCR5','MZB1','SDC1','IGHG1','IGHA1','IGHM','IGHD','PRDM1','XBP1','CD27','CD38','IGKC','IGLC2']
sc.pl.umap(adata,color=GOI,use_raw=False,color_map='inferno',vmin=0,vmax='p99',ncols=6,size=dotsize,frameon=False,show=False)
# plt.savefig('/Users/gouink/Documents/RTPD1Manuscript/Human/manuscript_review_analysis/analysis/'+celltype+'/GOI_markers_umap.png',dpi=600,bbox_inches='tight')

# for g in GOI:
#     sc.pl.umap(adata,color=g,use_raw=False,color_map='inferno',vmin=0,vmax='p99',size=dotsize,frameon=False,show=False)
#     plt.savefig('/Users/gouink/Documents/RTPD1Manuscript/Human/manuscript_review_analysis/analysis/'+celltype+'/'+g+'_umap.png',dpi=300,bbox_inches='tight')
#     plt.close()

In [ ]:
sc.pl.umap(adata,color=['AICDA','BCL6'],use_raw=False,color_map='inferno',vmin=0,vmax=1,ncols=6,size=dotsize*2,frameon=False,show=False)

In [ ]:
## fisher test for response heatmap

fig,axs,ratios,pvals = pf.fisher_response(adata[(adata.obs.pCR!='unknown')],groupby='leiden',clip=4,thresh=None,use_global=True,plot_umap=True)
plt.savefig('/Users/gouink/Documents/RTPD1Manuscript/Human/manuscript_review_analysis/analysis/'+celltype+'/fisher_response_normalizedtototal.png',dpi=600,bbox_inches='tight')

In [ ]:
## cluster dgex heatmap

dgex = pd.read_csv('/Users/gouink/Documents/RTPD1Manuscript/Human/manuscript_review_analysis/analysis/'+celltype+'/leiden_dgex.csv',index_col=0,header=0)
  
ax = dgexfunc.dgex_plot(adata,dgex=dgex,groupby='leiden',topn=50,pvalCutoff=0.05,fcCutoff=1,pctCutoff=0.3,use_FDR=True,
               dendro=False,plot_type='scanpy_heatmap',cmap='Blues',figsize=(2,1),fontsize=1)

# plt.savefig('/Users/gouink/Documents/RTPD1Manuscript/Human/manuscript_review_analysis/analysis/'+celltype+'/leiden_dgex_heatmap.svg')

In [ ]:
## boxplots of cluster percentages

fig,ax = pf.pct_boxplot(adata,groupby='leiden',rep='cohort',xcat='treatment',hcat='pCR',
                    xorder=['Base','PD1','RTPD1'],horder=['R','NR'],show_stats=True,thresh=None)

# plt.savefig('/Users/gouink/Documents/RTPD1Manuscript/Human/manuscript_review_analysis/analysis/'+celltype+'/leiden_pct_boxplot.svg')

In [ ]:
## lineplots of cluster percentages

fig,ax = pf.pct_lineplot(adata,groupby='leiden',rep='cohort',xcat='treatment',hcat='pCR',
                    xorder=['Base','PD1','RTPD1'],horder=['R','NR'],show_stats=True,direction='less',thresh=None)

# plt.savefig('/Users/gouink/Documents/RTPD1Manuscript/Human/manuscript_review_analysis/analysis/'+celltype+'/leiden_pct_lineplot.svg')

In [ ]:
## lineplots of cluster percentages

fig,ax = pf.med_lineplot(adata,groupby='leiden',rep='cohort',xcat='treatment',hcat='pCR',
                    xorder=['Base','PD1','RTPD1'],horder=['R','NR'],thresh=None)

# plt.savefig('/Users/gouink/Documents/RTPD1Manuscript/Human/manuscript_review_analysis/analysis/'+celltype+'/leiden_med_lineplot.svg')

In [ ]:
## correlation of B-cell cluster percentages with T-cell cluster percentages

df,ax = pf.prediction_query(adata=adata,metadata=None,
                             pred_group='leiden',pred_metric='percent',
                             target_cell='tcell_filtered',target_group='leiden',target_metric='percent',
                             thresh=10,drop_na=False,
                             return_df=True,plot=True,custom_heatmap=True,vmin=-1,vmax=1,fontsize=2,figsize=(2,2))

# plt.savefig('/Users/gouink/Documents/RTPD1Manuscript/Human/manuscript_review_analysis/analysis/'+celltype+'/bcell_tcell_cluster_correlation.svg')

In [ ]:
# ## correlation of B-cell cluster percentages with T-cell cluster percentages, plotted onto umap

# df = pf.prediction_query(adata=adata,metadata=None,
#                              pred_group='leiden',pred_metric='percent',
#                              target_cell='tcell_filtered',target_group='leiden',target_metric='percent',
#                              thresh=10,drop_na=False,
#                              return_df=True,plot=False,custom_heatmap=True,vmin=-1,vmax=1,fontsize=2,figsize=(2,2))

# df.dropna(how='any',inplace=True)
        
# cols = [x for x in df.columns if 'target' in x[0] or 'target' in x or 'pred' in x[0] or 'pred' in x]

# mat = df[cols].to_numpy()

# corr,pval = scipy.stats.spearmanr(mat,axis=0,nan_policy='raise')


# corr = pd.DataFrame(corr,index=cols,columns=cols)

# pval = -1*np.log10(pval)
# pval[np.isinf(pval)] = np.max(pval[~np.isinf(pval)])
# pval = pd.DataFrame(pval,index=cols,columns=cols)

# d = scipy.spatial.distance.pdist(corr.to_numpy(dtype='float64'),metric='euclidean')
# l = scipy.cluster.hierarchy.linkage(d,metric='euclidean',method='complete',optimal_ordering=True)
# dn = scipy.cluster.hierarchy.dendrogram(l,no_plot=True)
# order = dn['leaves']

# corr = corr.iloc[order,order]
# pval = pval.iloc[order,order]

clust = ['c4','c5','c6']
grps = adata.obs.leiden.unique().tolist()
fig,axs = plt.subplots(nrows=len(clust),ncols=1,figsize=(1.8,len(clust)*2))

from matplotlib import cm
norm = matplotlib.colors.Normalize(vmin=-1, vmax=1, clip=False)
mapper = cm.ScalarMappable(norm=norm, cmap=cm.RdBu_r)

corr2 = mapper.to_rgba(corr.to_numpy(dtype=np.float64))

for c,ax in zip(clust,axs.flat):
    
    tmp_corr = corr.loc[[x for x in corr.index if 'pred' in x[0]],[x for x in corr.columns if 'target' in x[0] and c in x[1]]]
    tmp_pval = pval.loc[[x for x in corr.index if 'pred' in x[0]],[x for x in corr.columns if 'target' in x[0] and c in x[1]]]

    for g in grps:
        
        row = np.argwhere(['pred' in x[0] and g in x[1] for x in corr.index]).item()
        col = np.argwhere(['target' in x[0] and c in x[1] for x in corr.columns]).item()

        
        xs = adata[(adata.obs.leiden==g)].obsm['X_umap'][:,0]
        ys = adata[(adata.obs.leiden==g)].obsm['X_umap'][:,1]
        size = pval.iloc[row,col].item()
        cs = corr2[row,col,:]
        
        _= ax.scatter(x=xs,y=ys,s=size/20,color=cs,linewidths=0.01,edgecolors='k')
        _= ax.grid(b=False)
        _= ax.set_xticks([])
        _= ax.set_yticks([])
        _= ax.set_title(c,fontdict={'fontsize':4},pad=0.1)
        
plt.savefig('/Users/gouink/Documents/RTPD1Manuscript/Human/manuscript_review_analysis/analysis/'+celltype+'/bcell_tcell_cluster_correlation_umap.png',dpi=600,bbox_inches='tight')

In [ ]:
## correlation of B-cell cluster percentages with myeloid cluster percentages

df,ax = pf.prediction_query(adata=adata,metadata=None,
                             pred_group='leiden',pred_metric='percent',
                             target_cell='myeloid_filtered',target_group='leiden',target_metric='percent',
                             thresh=10,drop_na=False,
                             return_df=True,plot=True,custom_heatmap=True,vmin=-1,vmax=1,fontsize=2,figsize=(2,2))

# plt.savefig('/Users/gouink/Documents/RTPD1Manuscript/Human/manuscript_review_analysis/analysis/'+celltype+'/bcell_myeloid_cluster_correlation.svg')

In [ ]:
## correlation of B-cell cluster percentages with myeloid cluster percentages, plotted on umap

df = pf.prediction_query(adata=adata,metadata=None,
                             pred_group='leiden',pred_metric='percent',
                             target_cell='myeloid_filtered',target_group='leiden',target_metric='percent',
                             thresh=10,drop_na=False,
                             return_df=True,plot=False,custom_heatmap=True,vmin=-1,vmax=1,fontsize=2,figsize=(2,2))

df.dropna(how='any',inplace=True)
        
cols = [x for x in df.columns if 'target' in x[0] or 'target' in x or 'pred' in x[0] or 'pred' in x]

mat = df[cols].to_numpy()

corr,pval = scipy.stats.spearmanr(mat,axis=0,nan_policy='raise')


corr = pd.DataFrame(corr,index=cols,columns=cols)

pval = -1*np.log10(pval)
pval[np.isinf(pval)] = np.max(pval[~np.isinf(pval)])
pval = pd.DataFrame(pval,index=cols,columns=cols)

d = scipy.spatial.distance.pdist(corr.to_numpy(dtype='float64'),metric='euclidean')
l = scipy.cluster.hierarchy.linkage(d,metric='euclidean',method='complete',optimal_ordering=True)
dn = scipy.cluster.hierarchy.dendrogram(l,no_plot=True)
order = dn['leaves']

corr = corr.iloc[order,order]
pval = pval.iloc[order,order]

clust = ['c11','c3']
grps = adata.obs.leiden.unique().tolist()
fig,axs = plt.subplots(nrows=len(clust),ncols=1,figsize=(1.8,len(clust)*2))

from matplotlib import cm
norm = matplotlib.colors.Normalize(vmin=-1, vmax=1, clip=False)
mapper = cm.ScalarMappable(norm=norm, cmap=cm.RdBu_r)

corr2 = mapper.to_rgba(corr.to_numpy(dtype=np.float64))

for c,ax in zip(clust,axs.flat):
    
    tmp_corr = corr.loc[[x for x in corr.index if 'pred' in x[0]],[x for x in corr.columns if 'target' in x[0] and c in x[1]]]
    tmp_pval = pval.loc[[x for x in corr.index if 'pred' in x[0]],[x for x in corr.columns if 'target' in x[0] and c in x[1]]]

    for g in grps:
        
        row = np.argwhere(['pred' in x[0] and g in x[1] for x in corr.index]).item()
        col = np.argwhere(['target' in x[0] and c in x[1] for x in corr.columns]).item()

        
        xs = adata[(adata.obs.leiden==g)].obsm['X_umap'][:,0]
        ys = adata[(adata.obs.leiden==g)].obsm['X_umap'][:,1]
        size = pval.iloc[row,col].item()
        cs = corr2[row,col,:]
        
        _= ax.scatter(x=xs,y=ys,s=size/20,color=cs,linewidths=0.01,edgecolors='k')
        _= ax.grid(b=False)
        _= ax.set_xticks([])
        _= ax.set_yticks([])
        _= ax.set_title(c,fontdict={'fontsize':4},pad=0.1)
        
plt.savefig('/Users/gouink/Documents/RTPD1Manuscript/Human/manuscript_review_analysis/analysis/'+celltype+'/bcell_myeloid_cluster_correlation_umap.png',dpi=600,bbox_inches='tight')

In [ ]:
## dgex of memory B-cell clusters with pathway enrichment analysis

## dgex
c = ['03','04','06']
r = ['00','01','02','05','07','08']

dgex = dgexfunc.custom_dgex(adata,comparator=c,reference=r,grouping_type='normal',groupby='leiden',bycluster=False)
dgex.to_csv('/Users/gouink/Documents/RTPD1Manuscript/Human/manuscript_review_analysis/analysis/'+celltype+'/memory_bcell_cluster_dgex.csv')

## pathway analysis
dgex = pd.read_csv('/Users/gouink/Documents/RTPD1Manuscript/Human/manuscript_review_analysis/analysis/'+celltype+'/memory_bcell_cluster_dgex.csv',index_col=0,header=0)

GOI = dgex[(dgex.pvals_adj<0.05)&(dgex.logfoldchanges>=1)].names.tolist()

genesets = ['/Users/gouink/.cache/gseapy/enrichr.GO_Biological_Process_2018.gmt']

enr_results = dgexfunc.pathway_enrich(rnk=GOI,genesets=genesets,test='enrich',background=adata.var_names.tolist())

## barplot of pathway analysis results
ax,tmp = dgexfunc.filter_pathways(enr_results,pval_cutoff=0.01,test='enrich')

plt.savefig('/Users/gouink/Documents/RTPD1Manuscript/Human/manuscript_review_analysis/analysis/'+celltype+'/memory_bcell_cluster_dgex_enrichr.svg')

In [ ]:
## IRE1-mediated unfolded protein response score

GOI = pd.read_csv('/Users/gouink/Documents/GeneLists/GO_IRE1mediatedUPR_0036498.csv',index_col=None,header=0)
sc.tl.score_genes(adata,gene_list=GOI.genename,use_raw=False,score_name='IRE1_UPR')
sc.pl.umap(adata,color='IRE1_UPR',color_map='viridis',size=dotsize,vmin='p01',vmax='p99',show=False)
plt.savefig('/Users/gouink/Documents/RTPD1Manuscript/Human/manuscript_review_analysis/analysis/'+celltype+'/IRE1_UPR_score_umap.png',dpi=300,bbox_inches='tight')

In [ ]:
GOI = ['ANKRD28','CYTOR','CD38','PRDM1','XBP1','MZB1','SSR4']
sc.pl.umap(adata,color=GOI,use_raw=False,color_map='inferno',vmin=0,vmax='p99',ncols=3,size=dotsize,frameon=False,show=False)